In [ ]:
!pip install pytket

In [ ]:
import os
from os.path import join, isfile
import csv

from pytket import Circuit, OpType
from pytket.qasm import circuit_from_qasm, circuit_to_qasm
from pytket.passes import FullPeepholeOptimise, RebaseCustom

In [ ]:
# TK1 → Rz-SX-Rz
def tk1_replacement(a, b, c) -> Circuit:
    circ = Circuit(1)
    circ.Rz(float(a), 0)
    circ.add_gate(OpType.SX, [0])
    circ.Rz(float(c), 0)
    return circ


def cx_replacement() -> Circuit:
    circ = Circuit(2)
    circ.CX(0, 1)
    return circ


def tket_optimize_nam(fn: str, output_dir: str):
    circ = circuit_from_qasm(fn)

    FullPeepholeOptimise().apply(circ)

    rebase_pass = RebaseCustom(
        gateset={OpType.CX, OpType.Rz, OpType.SX, OpType.X},
        cx_replacement=cx_replacement(),
        tk1_replacement=tk1_replacement,
    )

    rebase_pass.apply(circ)

    output_path = os.path.join(output_dir, os.path.basename(fn))
    circuit_to_qasm(circ, output_path)

    return circ.n_gates, circ.n_2qb_gates(), circ.depth()


circuit_order = [
    "tof_3",
    "barenco_tof_3",
    "mod5_4",
    "tof_4",
    "tof_5",
    "barenco_tof_4",
    "mod_mult_55",
    "vbe_adder_3",
    "barenco_tof_5",
    "csla_mux_3",
    "rc_adder_6",
    "gf2^4_mult",
    "hwb6",
    "mod_red_21",
    "tof_10",
    "gf2^5_mult",
    "csum_mux_9",
    "barenco_tof_10",
    "ham15-low",
    "qcla_com_7",
    "gf2^6_mult",
    "qcla_adder_10",
    "gf2^7_mult",
    "gf2^8_mult",
    "qcla_mod_7",
    "adder_8",
    "vqe_nativegates_ibm_tket_8",
    "qgan_nativegates_ibm_tket_8",
    "qaoa_nativegates_ibm_tket_8",
    "ae_nativegates_ibm_tket_8",
    "qpeexact_nativegates_ibm_tket_8",
    "qpeinexact_nativegates_ibm_tket_8",
    "qft_nativegates_ibm_tket_8",
    "qftentangled_nativegates_ibm_tket_8",
    "portfoliovqe_nativegates_ibm_tket_8",
    "portfolioqaoa_nativegates_ibm_tket_8",
]

path = "../circuits/ibm/"
fns = [
    fn for fn in os.listdir(path) if isfile(join(path, fn)) and fn[-4:] == "qasm"
]

extra_path = "../circuits/ibm_extra/"
extra_fns = [
    fn
    for fn in os.listdir(extra_path)
    if isfile(join(extra_path, fn)) and fn[-4:] == "qasm"
]

fns = fns + extra_fns
paths = [path] * len(fns) + [extra_path] * len(extra_fns)

output_dir = "./output/tket_ibm/"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

result_dict = {}

for i, (fn, path) in enumerate(zip(fns, paths)):
    if fn[:-5] not in circuit_order:
        continue
    full_path = path + fn
    gate_cnt, cnot_cnt, depth = tket_optimize_nam(
        full_path, output_dir
    )
    result_dict[fn] = [gate_cnt, cnot_cnt, depth]

csv_file = "./metricks/tket_ibm.csv"

with open(csv_file, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "total_gates", "cx_gates", "depth"])
    for fn in circuit_order:
        if fn + ".qasm" in result_dict:
            writer.writerow([fn + ".qasm"] + result_dict[fn + ".qasm"])